## Concise: 1: Start
Author: yanjun zhang, Source from book "Abali - 2017 - Computational Reality" P119

In [1]:
from disc_f import *
setup_simulation()
import time
from disc_f import *
# calculate how long time the simulation it is
start_time = time.time()

# mesh-size, contact area coefficient
mesh_min = 3
mesh_max = 15
c_contact = 1

# Each time step rotation angular, and acc during lag, 1 is full acc, 0 is no acc.
angular_r = 224000/2
v_vehicle = 160
c_acc = 1

# calling local functions to get all parameters
(dt, P, g, num_steps, h, radiation, v_angular, Ti, Tm, S_rub_circle, t, rho, c, k, t_brake, 
S_total,) = vehicle_initial (angular_r, v_vehicle, c_contact, c_acc)

print("1: Total tims is ", round(sum(dt), 2), "s")
print("2: Total numb steps is ", num_steps)


DOLFINx version: 0.8.0
Simulation environment setup complete.
1: Total tims is  -119.47 s
2: Total numb steps is  3


## 2: Mesh

In [2]:
domain, cell_markers, facet_amrkers, \
mesh_name, mesh_filename1, mesh_filename2 = mesh_brake_all(mesh_min,mesh_max)

# Define variational problem, CG is Lagrabge
V = fem.functionspace(domain, ("CG", 1))

# u_n is for initial condition and uh is the solver result.
# variable, need to be projected form Q onto V, DG is discontinuous Lagrange. 
Q = fem.functionspace(domain, ("DG", 0))
T_init = Function(Q)
T_init.name = "u_n"
T_init.x.array[:] = np.full_like(1, Ti, dtype=default_scalar_type)
u_n = project(T_init, V)
u_n.name = "u_n"

fdim = domain.topology.dim - 1

## bc_disc is zero, no any dirichlete boundary condition, z = 100, not exist
bc_disc = mesh.locate_entities_boundary(domain, fdim, lambda x: np.isclose(x[2], 100)) 
bc = fem.dirichletbc( PETSc.ScalarType(Tm), fem.locate_dofs_topological(V, fdim, bc_disc), V  )
np.set_printoptions(threshold=np.inf)

import meshio
mesh1 = meshio.read(mesh_filename1)
total_elements = sum(len(cells.data) for cells in mesh1.cells)

# create xdmf file, for paraview
xdmf_name = "T-s-{}-d-{}-{}-c-{}-e-{}.xdmf".format( num_steps, angular_r, mesh_filename2, c_contact, total_elements)
h5_name = "T-s-{}-d-{}-{}-c-{}-e-{}.h5".format( num_steps, angular_r, mesh_filename2, c_contact, total_elements )

xdmf = io.XDMFFile(domain.comm, xdmf_name, "w")
xdmf.write_mesh(domain)

# Create boundary condition
x_co, y_co = get_rub_coordinate()
common_indices3, facet_markers3, sorted_indices3 = target_facets (domain, x_co, y_co, S_rub_circle )
facet_tag = meshtags (domain, fdim, common_indices3[sorted_indices3], facet_markers3[sorted_indices3] )
ds = Measure("ds", domain=domain, subdomain_data=facet_tag)


The file 'm-3-15.msh' does not exist, start building:
Info    : Meshing 1D...                                                                                                                 
Info    : [  0%] Meshing curve 4 (Circle)
Info    : [ 10%] Meshing curve 5 (Line)
Info    : [ 10%] Meshing curve 6 (Circle)
Info    : [ 10%] Meshing curve 7 (Circle)
Info    : [ 10%] Meshing curve 8 (Line)
Info    : [ 10%] Meshing curve 9 (Circle)
Info    : [ 10%] Meshing curve 10 (Circle)
Info    : [ 10%] Meshing curve 11 (Line)
Info    : [ 20%] Meshing curve 12 (Circle)
Info    : [ 20%] Meshing curve 13 (Line)
Info    : [ 20%] Meshing curve 14 (Line)
Info    : [ 20%] Meshing curve 15 (Circle)
Info    : [ 20%] Meshing curve 16 (Line)
Info    : [ 20%] Meshing curve 17 (Line)
Info    : [ 20%] Meshing curve 18 (Line)
Info    : [ 30%] Meshing curve 19 (Line)
Info    : [ 30%] Meshing curve 20 (Circle)
Info    : [ 30%] Meshing curve 21 (Circle)
Info    : [ 30%] Meshing curve 22 (Circle)
Info    : [ 30%]

Info    : [ 30%] Meshing curve 25 (Circle)
Info    : [ 40%] Meshing curve 26 (Circle)
Info    : [ 40%] Meshing curve 27 (Circle)
Info    : [ 40%] Meshing curve 28 (Circle)
Info    : [ 40%] Meshing curve 29 (Circle)
Info    : [ 40%] Meshing curve 30 (Circle)
Info    : [ 40%] Meshing curve 31 (Circle)
Info    : [ 40%] Meshing curve 32 (Circle)
Info    : [ 50%] Meshing curve 33 (Circle)
Info    : [ 50%] Meshing curve 34 (Circle)
Info    : [ 50%] Meshing curve 35 (Circle)
Info    : [ 50%] Meshing curve 36 (Circle)
Info    : [ 50%] Meshing curve 37 (Circle)
Info    : [ 50%] Meshing curve 38 (Circle)
Info    : [ 50%] Meshing curve 39 (Line)
Info    : [ 60%] Meshing curve 40 (Line)
Info    : [ 60%] Meshing curve 41 (Circle)
Info    : [ 60%] Meshing curve 42 (Line)
Info    : [ 60%] Meshing curve 43 (Circle)
Info    : [ 60%] Meshing curve 44 (Line)
Info    : [ 60%] Meshing curve 45 (Circle)
Info    : [ 60%] Meshing curve 46 (Line)
Info    : [ 60%] Meshing curve 47 (Circle)
Info    : [ 70%] Mesh

NameError: name 'fem' is not defined

## 3: Variational equation


In [ ]:
uh = fem.Function(V)
uh.name = "uh"
uh = project(T_init, V)  ##give temperature to all elements
t = 0
xdmf.write_function(uh, t)

# u = trial function, solution what we want to know
u = fem.Function(V)
v = ufl.TestFunction(V)
f = fem.Constant(domain, PETSc.ScalarType(0))  ## heat source is 0
n_vector = FacetNormal(domain)

F = ( (rho * c) / dt[0] * inner(u, v) * dx
    + k * inner(grad(u), grad(v)) * dx
    + h * inner(u, v) * ds(200)    #200 is the contact area, details are in function of target_facets.
    + radiation * inner(u**4, v) * ds(200)
    - ( inner(f, v) * dx
        + (rho * c) / dt[0] * inner(u_n, v) * dx
        + h * Tm * v * ds(200)
        + radiation * (Tm**4) * v * ds(200) ) )

for i in list(range(1, 19)):  # before 2024/5/16
    F += (+ inner(g[0], v) * ds(10 * i) 
          - h * inner( u, v) * ds(10 * i)  
          - radiation * inner( (u**4 - Tm**4), v) * ds(10 * i)  )

problem = NonlinearProblem(F, u, bcs=[bc])
n,converged = solver_setup_solve(problem,u);

## 8:Visualization of time dependent problem using pyvista
gif_name = "T-s-{}-d-{}-{}-c-{}-e-{}.gif".format(
    num_steps, angular_r, mesh_filename2, c_contact, total_elements)

plotter, sargs, renderer, warped, viridis, grid = plot_gif(V,u,gif_name)

## 4: Solution


In [ ]:
T_array = [(0, [Ti for _ in range(len(u.x.array))])]
total_degree = 0

for i in range(num_steps):
    t += dt[i]

    x_co, y_co = rub_rotation(x_co, y_co, angular_r)  # update the location
    total_degree += angular_r  # Incrementing degree by 10 in each step

    sys.stdout.write("\r1: Rotation has applied for {} degree. ".format(total_degree))
    sys.stdout.write("2: Current time is " + str(round(t, 1)) + " s. ")
    sys.stdout.write("3: Completion is " + str(round(100 * (t / t_brake), 1)) + " %. ")
    sys.stdout.flush()

    common_indices3, facet_markers3, sorted_indices3 = target_facets(
                domain, x_co, y_co, S_rub_circle  )
    facet_tag = meshtags( domain, fdim, common_indices3[sorted_indices3], \
                         facet_markers3[sorted_indices3]  )
    ds = Measure("ds", domain=domain, subdomain_data=facet_tag)

    F = ((rho * c) / dt[i] * inner(u, v) * dx
        + k * inner(grad(u), grad(v)) * dx
        + h * inner(u, v) * ds(200)
        + radiation * inner(u**4, v) * ds(200)
        - ( inner(f, v) * dx
            + (rho * c) / dt[i] * inner(u_n, v) * dx
            + h * Tm * v * ds(200)
            + radiation * (Tm**4) * v * ds(200)) )

    for j in list(range(1, 19)):
        #F += -k * dot(grad(u) * v, n_vector) * ds(10 * j) - inner(g[i], v) * ds(10 * j)
        F += ( - inner(g[i], v) * ds(10 * j) 
               - h * inner( u, v) * ds(10 * j)  
               - radiation * inner( (u**4 - Tm**4), v) * ds(10 * j) )    

    problem = NonlinearProblem(F, u, bcs=[bc])

    ## 7: Using petsc4py to create a linear solver
    solver_setup_solve(problem,u)
    u.x.scatter_forward()

    sys.stdout.write("1: Completion is " + str(round(100 * (t / t_brake), 1)) + " %. ")
    sys.stdout.flush()     

    # Update solution at previous time step (u_n)
    u_n.x.array[:] = u.x.array
    T_array.append((t, u.x.array.copy()))
    # Write solution to file
    xdmf.write_function(u, t)
    # Update plot
    #warped = grid.warp_by_scalar("uh", factor=0)
    plotter.update_coordinates(warped.points.copy(), render=False)
    plotter.update_scalars(u.x.array, render=False)
    plotter.write_frame()

plotter.close()
xdmf.close()
print()

csv_name = "Result_T-s-{}-d-{}-{}-c-{}-e-{}.csv".format(
    num_steps, angular_r, mesh_filename2, c_contact, total_elements  )

save_t_T(csv_name, T_array)


In [ ]:
from IPython.display import display, Image
display(Image(gif_name))

# 6: Post process

In [ ]:
from disc_f import *
mesh_min = 3
mesh_max = 15
domain_pad, cell_markers_pad, facet_amrkers_pad,mesh_name_pad,\
mesh_filename1_pad, mesh_filename2_pad = mesh_brake_pad1  ( mesh_min,mesh_max)

In [ ]:
## here domain is the whole brake pad and disc, but we only need for brake pad
VT = fem.functionspace(domain_pad, ("CG", 1))              #define the finite element function space

T_, dT = ufl.TestFunction(VT), ufl.TrialFunction(VT)   # T_ is the test function, like v
Delta_T = fem.Function(VT, name="Temperature_variation")
aT = ufl.dot(ufl.grad(dT), ufl.grad(T_)) * ufl.dx      # a is grad T, grad T_, like grad u, grad v
LT = fem.Constant(domain_pad, 0.0) * T_ * ufl.dx           # L is 0, o*v, v is T_
bc_disc_T = mesh.locate_entities_boundary(domain_pad, fdim, lambda x: np.isclose(x[2], 50))

def contact(x):
    return np.isclose(x[2],50)
    
contact_dofs_T = fem.locate_dofs_geometrical(VT, contact)  
bcT= [ fem.dirichletbc( 64.0, contact_dofs_T, VT )  ]

problem = fem.petsc.LinearProblem(aT, LT, u=Delta_T, bcs=bcT)
problem.solve()

In [ ]:
#######try to make domain only for brake pad.

E = fem.Constant(domain_pad, 50e3)             # Elastic module
nu = fem.Constant(domain_pad, 0.2)             # Poission ratio
gdim = domain_pad.geometry.dim

mu = E / 2 / (1 + nu)                      # Shear modulus
lmbda = E * nu / (1 + nu) / (1 - 2 * nu)   # Lame parameters
alpha = fem.Constant(domain_pad, 1e-5)         # Thermal expansion coefficient

f1 = fem.Constant(domain_pad, (0.0, 0.0, 0.0))       # O for external force

def eps(v):                                # epsilon, strain, the deforamtion, dy/y 
    return ufl.sym(ufl.grad(v))

def sigma(v, Delta_T):                     # sigmathis is sigma
    return (lmbda * ufl.tr(eps(v)) - alpha * (3 * lmbda + 2 * mu) * Delta_T 
    ) * ufl.Identity(gdim)  + 2.0 * mu * eps(v)     # here braces is important, can not be in above line

Vu = fem.functionspace(domain_pad, ("CG", 1, (gdim,))) 
du = ufl.TrialFunction(Vu)
u_ = ufl.TestFunction(Vu)

Wint = ufl.inner(sigma(du, Delta_T), eps(u_)) * ufl.dx  # here du is unkown
aM = ufl.lhs(Wint)                                      # Wint is long and lhs can help to distinguish unkown and know.
LM = ufl.rhs(Wint) + ufl.inner(f1, u_) * ufl.dx          # knows parameters are in lhs

def up_side(x):
    return np.logical_or(np.isclose(x[2], 53), np.isclose(x[2], 83))
    
up_dofs_u = fem.locate_dofs_geometrical(Vu, up_side) # lateral sides of domain
bcu = [fem.dirichletbc(np.zeros((gdim,)), up_dofs_u, Vu)]  # displacement Vu is fixed in lateral sides

u = fem.Function(Vu, name="Displacement")
problem = fem.petsc.LinearProblem(aM, LM, u=u, bcs=bcu)
problem.solve()

In [ ]:
u_topology, u_cell_types, u_geometry = plot.vtk_mesh(Vu)                # get mesh data
u_grid = pyvista.UnstructuredGrid(u_topology, u_cell_types, u_geometry) # plot grid
u_3D = np.zeros((u_geometry.shape[0], 3))
u_3D[:, :3] = u.x.array.reshape(-1, 3)
u_grid.point_data["Displacement_thermal"] = u_3D
u_grid.set_active_vectors("Displacement_thermal")
warped = u_grid.warp_by_vector("Displacement_thermal", factor=10000000)

plotter = pyvista.Plotter()
plotter.window_size = (800, 300)
plotter.add_mesh(warped)
edges = warped.extract_all_edges()
plotter.add_mesh(edges, color="k", line_width=1, opacity=0.5)
plotter.view_xy()
plotter.camera.elevation = 180  # Rotate the camera around the Y-axis
plotter.camera.roll= 135

plotter.zoom_camera(2)
plotter.show()